In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import random
                                                       

## Ranking des critères

Les trois critères de décision vont être hiérarchisés pour chacun des 10 décideurs présents à l'assemblée pour décider du futur planning.

In [95]:
def pref_aleatoire(liste_employes):
    dict_emp = {}
    for employe in liste_employes:
        liste_criteres = ['Benefice','projets max','longueur max']
        random.shuffle(liste_criteres)
        dict_emp[employe]= liste_criteres
    return dict_emp
        
    

In [96]:
# On fixe le choix des décideurs au premier random effectué
decideurs = ['Alice','Bob','Charlie','Dan','Erin','Frank','Meco','Lulu','Faufau','Tango','Thomas']
#dico_preference = pref_aleatoire(decideurs)
#dico_preference

In [97]:
dico_preference = {'Alice': ['longueur max', 'projets max', 'Benefice'],
 'Bob': ['longueur max', 'projets max', 'Benefice'],
 'Charlie': ['Benefice', 'projets max', 'longueur max'],
 'Dan': ['longueur max', 'Benefice', 'projets max'],
 'Erin': ['projets max', 'Benefice', 'longueur max'],
 'Frank': ['Benefice', 'longueur max', 'projets max'],
 'Meco': ['longueur max', 'projets max', 'Benefice'],
 'Lulu': ['projets max', 'Benefice', 'longueur max'],
 'Faufau': ['projets max', 'Benefice', 'longueur max'],
 'Tango': ['projets max', 'Benefice', 'longueur max'],
 'Thomas': ['longueur max', 'projets max', 'Benefice']}

In [98]:
# Choix de la taille des instances

taille = "small"
#taille = "medium"
#taille = "large"


In [85]:
# path = "/Users/macoco/Desktop/CS/3A/SDP/projet_sdp/project-sdp/results/final"
path = "results/final"

df = pd.read_excel(path+'/'+taille+'/res.xlsx')
#header=[0,1],index_col=[0]
#df = df.iloc[2:]
df = df.drop(columns=['Unnamed: 0'])
df = df.drop_duplicates()
df

,Benefice,longueur max,projets max
0,65,4,4
1,65,4,3
2,65,4,2
3,65,4,1
4,55,3,1
8,40,2,1
12,20,1,1


In [86]:
#Dropping dominated solutions : 

#Small
df = df.drop(df.index[0:3], axis=0)
df

#Medium
#df = df.drop(df.index[0:13], axis=0)

#Large
#df = df.drop(df.index[0:23], axis=0)
#df = df.drop([792], axis=0)


,Benefice,longueur max,projets max
3,65,4,1
4,55,3,1
8,40,2,1
12,20,1,1


In [87]:
initial_candidates = df.index.values.tolist()
initial_candidates

[3, 4, 8, 12]

In [88]:
dict_initial_candidates = {}
for i in initial_candidates :
    dict_initial_candidates[i]=0
dict_initial_candidates

{3: 0, 4: 0, 8: 0, 12: 0}

In [89]:
votes = []
for decideur in decideurs :
    dico_vote={}
    
    tf_0 = 'Benefice'!=dico_preference[decideur][0]
    tf_1 = 'Benefice'!=dico_preference[decideur][1]
    tf_2 = 'Benefice'!=dico_preference[decideur][2]
    df_ranked = df.sort_values([dico_preference[decideur][0],
                                dico_preference[decideur][1],
                                dico_preference[decideur][2]],ascending=[tf_0,tf_1,tf_2])
    #dico_ranking[decideur]=df_ranked.index.values.tolist()
    candidates = df_ranked.index.values.tolist()
    dict_initial_candidates[candidates[0]]+=1
    for i in range(len(candidates)) : 
        dico_vote[candidates[i]]=i+1
    votes.append(dico_vote)
    
votes

    
    

[{12: 1, 8: 2, 4: 3, 3: 4},
 {12: 1, 8: 2, 4: 3, 3: 4},
 {3: 1, 4: 2, 8: 3, 12: 4},
 {12: 1, 8: 2, 4: 3, 3: 4},
 {3: 1, 4: 2, 8: 3, 12: 4},
 {3: 1, 4: 2, 8: 3, 12: 4},
 {12: 1, 8: 2, 4: 3, 3: 4},
 {3: 1, 4: 2, 8: 3, 12: 4},
 {3: 1, 4: 2, 8: 3, 12: 4},
 {3: 1, 4: 2, 8: 3, 12: 4},
 {12: 1, 8: 2, 4: 3, 3: 4}]

In [90]:
dict_initial_candidates

{3: 6, 4: 0, 8: 0, 12: 5}

## Choix des systèmes de vote

### Gagnant de Condorcet

In [93]:
import condorcet

evaluator = condorcet.CondorcetEvaluator(candidates=candidates, votes=votes)
winner, rest_of_table = evaluator.get_n_winners(1)
print(winner)
print(df.loc[winner])


[3]
   Benefice  longueur max  projets max
3        65             4            1


In [94]:
rest_of_table

{8: {'wins': [12], 'losses': [4]},
 12: {'wins': [], 'losses': [8, 4]},
 4: {'wins': [12, 8], 'losses': []}}

### Results Condorcet : 

**small** : planning **3** est le gagnant de Condorcet \
**medium** : planning **13** est le gagnant de Condorcet \
**large** : planning **23** est le gagnant de Condorcet


### Positional Scoring Rules (Borda)

In [73]:
score_borda = {}
for candidate in initial_candidates :
    score_candidate = 0
    for dico in votes :
        score_candidate += dico[candidate]
    score_borda[candidate]=score_candidate

score_borda

print(min(score_borda, key=score_borda.get))
print(score_borda)
        

3
{3: 26, 4: 27, 8: 28, 12: 29}


### Results Borda : 

**small** : planning **3** est le gagnant de Borda \
**medium** : planning **13** est le gagnant de Borda \
**large** : planning **23** est le gagnant de Borda


### Plurality with instant Run-off

In [74]:
import pyrankvote
from pyrankvote import Candidate, Ballot

candidates_instant_run_off = []
for i in initial_candidates:
    candidates_instant_run_off.append(Candidate(str(i)))
    
liste_votes_decideurs = []
for decideur in decideurs :
    vote_pyrank = []

    tf_0 = 'Benefice'!=dico_preference[decideur][0]
    tf_1 = 'Benefice'!=dico_preference[decideur][1]
    tf_2 = 'Benefice'!=dico_preference[decideur][2]
    df_ranked = df.sort_values([dico_preference[decideur][0],
                                dico_preference[decideur][1],
                                dico_preference[decideur][2]],ascending=[tf_0,tf_1,tf_2])
    #dico_ranking[decideur]=df_ranked.index.values.tolist()
    candidates = df_ranked.index.values.tolist()
    for i in candidates : 
        vote_pyrank.append(Candidate(str(i)))

    liste_votes_decideurs.append(vote_pyrank)

ballots = []
for liste in liste_votes_decideurs :
    ballots.append(Ballot(ranked_candidates=liste))

In [75]:
# You can use your own Candidate and Ballot objects as long as they implement the same properties and methods
election_result = pyrankvote.instant_runoff_voting(candidates_instant_run_off, ballots)

winners = election_result.get_winners()

print(election_result)

FINAL RESULT
  Candidate    Votes  Status
-----------  -------  --------
          3        6  Elected
         12        5  Rejected
          4        0  Rejected
          8        0  Rejected



### Results Plurality with instant run-off : 

**small** : le planning **3** est vainqueur au premier tour \
**medium** : le planning **13** est vainqueur au premier tour \
**large** : planning **23** est le gagnant de Borda


Plusieurs itération du système de décision

In [57]:
def iteration_votes(taille):
    ## CHARGEMENT DES DECIDEURS
    decideurs = ['Alice','Bob','Charlie','Dan','Erin','Frank','Meco','Lulu','Faufau','Tango','Thomas']
    dico_preference = pref_aleatoire(decideurs)
    df = pd.read_excel('results/final/'+taille+'/res.xlsx')
    df = df.drop(columns=['Unnamed: 0'])
    df = df.drop_duplicates()
    initial_candidates = df.index.values.tolist()
    dict_initial_candidates = {}
    for i in initial_candidates :
        dict_initial_candidates[i]=0

    ## PROCESSUS DE PREFERENCE PAR LEADER
    votes = []
    for decideur in decideurs :
        dico_vote={}
        
        tf_0 = 'Benefice'!=dico_preference[decideur][0]
        tf_1 = 'Benefice'!=dico_preference[decideur][1]
        tf_2 = 'Benefice'!=dico_preference[decideur][2]
        df_ranked = df.sort_values([dico_preference[decideur][0],
                                    dico_preference[decideur][1],
                                    dico_preference[decideur][2]],ascending=[tf_0,tf_1,tf_2])
        candidates = df_ranked.index.values.tolist()
        dict_initial_candidates[candidates[0]]+=1
        for i in range(len(candidates)) : 
            dico_vote[candidates[i]]=i+1
        votes.append(dico_vote)

    ## VOTE DE CONDORCET
    evaluator = condorcet.CondorcetEvaluator(candidates=candidates, votes=votes)
    winner_condorcet, rest_of_table = evaluator.get_n_winners(1)

    ## VOTE DE BORDA
    score_borda = {}
    for candidate in initial_candidates :
        score_candidate = 0
        for dico in votes :
            score_candidate += dico[candidate]
        score_borda[candidate]=score_candidate
    winner_borda = min(score_borda, key=score_borda.get)

    ## VOTE AVEC RUN-OFF
    candidates_instant_run_off = []
    for i in initial_candidates:
        candidates_instant_run_off.append(Candidate(str(i)))
        
    liste_votes_decideurs = []
    for decideur in decideurs :
        vote_pyrank = []

        tf_0 = 'Benefice'!=dico_preference[decideur][0]
        tf_1 = 'Benefice'!=dico_preference[decideur][1]
        tf_2 = 'Benefice'!=dico_preference[decideur][2]
        df_ranked = df.sort_values([dico_preference[decideur][0],
                                    dico_preference[decideur][1],
                                    dico_preference[decideur][2]],ascending=[tf_0,tf_1,tf_2])
        candidates = df_ranked.index.values.tolist()
        for i in candidates : 
            vote_pyrank.append(Candidate(str(i)))

        liste_votes_decideurs.append(vote_pyrank)

    ballots = []
    for liste in liste_votes_decideurs :
        ballots.append(Ballot(ranked_candidates=liste))


    election_result = pyrankvote.instant_runoff_voting(candidates_instant_run_off, ballots)
    winners_runoff = election_result.get_winners()

    return [winner_condorcet, winner_borda, winners_runoff]
    

In [58]:
res = {'Condorcet': [], 'Borda': [], 'Runoff':[]}

for _ in range(10):
    iteration = iteration_votes(taille)
    res['Condorcet'].append(iteration[0])
    res['Borda'].append(iteration[1])
    res['Runoff'].append(iteration[2])

print(res)
## traitement stat, peut être faire un mapping ?

{'Condorcet': [[280], [13], [280], [280], [280], [280], [13], [280], [13], [280]], 'Borda': [13, 13, 280, 280, 13, 13, 13, 13, 13, 13], 'Runoff': [[<Candidate('280')>], [<Candidate('13')>], [<Candidate('280')>], [<Candidate('280')>], [<Candidate('280')>], [<Candidate('280')>], [<Candidate('13')>], [<Candidate('280')>], [<Candidate('13')>], [<Candidate('280')>]]}


# Scores des objectifs :

3: Bénéfice = 65 ; longueur_max = 4 ; projet_max = 1 \
13 : Bénéfice = 413 ; longueur_max = 21 ; projet_max = 1 \
23 : Bénéfice = 817 ; longueur_max = 35 ; projet_max = 1